### Melon 100 Chart 
* 100곡의 노래의 제목, Song ID추출
* Song ID로 노래의 상세정보를 100 번 요청
* 100곡의 정보를 json에 저장
* json file을 읽어서 Pandas DataFrame에 저장해서 표데이터로 만들기
* 표데이터를 DB에 저장

In [41]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
}
url = 'https://www.melon.com/chart/index.htm'

res = requests.get(url, headers=req_header)
#print(res.status_code)
soup = BeautifulSoup(res.text, 'html.parser')
# print(len(soup.select("a[href*='playSong']")))
# print(len(soup.select('div#tb_list'))) #1개
# print(len(soup.select('div#tb_list tr'))) #101
# print(len(soup.select('div#tb_list tr div.wrap_song_info'))) #200
# print(len(soup.select('div#tb_list tr div.wrap_song_info a'))) #427
# print(len(soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']"))) #100

#100곡의 song 정보를 저장할 list 선언
song_list = list()

atags = soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']")
for idx,atag in enumerate(atags,1):
    #1곡의 song 정보를 저장할 dict 선언
    song_dict = dict()
    title = atag.text
    song_dict['title'] = title
    href = atag['href']
    matched = re.search(r'(\d+)\);', href)
    song_id = matched.group(1)
    song_dict['id'] = song_id
    song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
    #print(idx,title, song_url)
    song_dict['url'] = song_url
    #print(idx, song_dict)
    song_list.append(song_dict)

print(len(song_list))    
song_list[0:4]

100


[{'title': 'STAY',
  'id': '33658563',
  'url': 'https://www.melon.com/song/detail.htm?songId=33658563'},
 {'title': '신호등',
  'id': '33480898',
  'url': 'https://www.melon.com/song/detail.htm?songId=33480898'},
 {'title': 'My Universe',
  'id': '33998510',
  'url': 'https://www.melon.com/song/detail.htm?songId=33998510'},
 {'title': 'Permission to Dance',
  'id': '33666269',
  'url': 'https://www.melon.com/song/detail.htm?songId=33666269'}]

### 100곡의 Song의 상세정보 파싱하기

In [47]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
}
#100곡의 Song 상세정보를 저장할 list 선언
song_detail_list = []
for idx,song in enumerate(song_list[0:3],1):
    #1곡의 Song 상세정보를 저장할 dict 선언
    song_detail_dict = {}
    res = requests.get(song['url'], headers=req_header)
    soup = BeautifulSoup(res.text, 'html.parser')
    #노래제목
    song_detail_dict['곡명'] = song['title']
    #가수
    song_detail_dict['가수'] = soup.select("a[href*='goArtistDetail'] span")[0].text
    #앨범
    song_detail_dict['앨범'] = soup.select('div.meta dd')[0].text
    #발매일
    song_detail_dict['발매일'] = soup.select('div.meta dd')[1].text
    #장르
    song_detail_dict['장르'] = soup.select('div.meta dd')[2].text
    #좋아요 건수
    song_id = song['id']
    like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
    like_res = requests.get(like_url, headers=req_header)
    #print(song['title'], like_res.json()['contsLike'][0]['SUMMCNT'])
    song_detail_dict['좋아요'] = like_res.json()['contsLike'][0]['SUMMCNT']
    #가사
    lyric_tag = soup.select('div#d_video_summary')
    if lyric_tag:
        lyric = lyric_tag[0].text
        #print(lyric)
    else:
        lyric = ''
    
    song_detail_dict['가사'] = lyric
    print(song_detail_dict)
    


							I do the same thing I told youthat I never wouldI told you I'd changeeven when I knew I never couldI know that I can't findnobody elseas good as youI need you to stayneed you to stay hey OhI get drunk wake upI'm wasted stillI realize the timethat I wasted hereI feel like you can'tfeel the way I feelOh I'll be fucked upif you can't be right hereOh ooh-woahOh ooh-woah ooh-woahOh ooh-woahOh ooh-woah ooh-woahOh ooh-woahOh ooh-woah ooh-woahOh I'll be fucked upif you can't be right hereI do the same thing I told youthat I never wouldI told you I'd changeeven when I knew I never couldI know that I can't findnobody elseas good as youI need you to stayneed you to stay heyI do the same thing I told youthat I never wouldI told you I'd changeeven when I knew I never couldI know that I can't findnobody elseas good as youI need you to stayneed you to stay heyWhen I'm away from youI miss your touch OohYou're the reasonI believe in love OohIt's been difficultfor me to trust OohAnd I'm afraidth